In [ ]:
from google.colab import userdata
username = userdata.get('username')
key = userdata.get('key')
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c image-search -p Dataset
!unzip "Dataset/*.zip" -d Dataset
!rm Dataset/*.zip

# Import zone

In [ ]:
import os
import pandas as pd
from PIL import Image, ImageOps, ImageFilter
import torch
from transformers import AutoModel, CLIPProcessor
from tqdm import tqdm
import cv2
import numpy as np

# Image preprocess

In [ ]:
def process_image(image_path, model, processor):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((224, 224))

    #denoise
    image = ImageOps.autocontrast(image)
    image = cv2.fastNlMeansDenoisingColored(np.array(image),None,20,20,7,21)

    image = Image.fromarray(image)

    inputs = processor(images=image, return_tensors='pt').to('cuda')
    outputs = model.get_image_features(inputs.pixel_values).cpu()

    outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)

    return outputs

In [ ]:
def process_query_images(query_dir, model, processor):
    query_images = []
    query_classes = []

    with torch.no_grad():
        for file in os.listdir(query_dir):
            image_path = os.path.join(query_dir, file)
            outputs = process_image(image_path, model, processor)
            query_images.append(outputs)
            query_classes.append(int(file[:-4]))

    query_images = torch.cat(query_images)
    return query_images, query_classes

In [ ]:
def find_top_match(image_path, model, processor, query_images, query_classes):
    image = Image.open(image_path).convert('RGB')

    image = image.resize((224, 224))
    image = image.filter(ImageFilter.BLUR)

    image = cv2.fastNlMeansDenoisingColored(np.array(image),None,20,20,7,21)
    image = Image.fromarray(image)

    inputs = processor(images=image, return_tensors='pt').to('cuda')
    outputs = process_image(image_path, model, processor)

    cosine = torch.cosine_similarity(outputs, query_images)

    if cosine.max() > 0.8351:
        return query_classes[cosine.argmax().numpy().tolist()]
    else:
        return None

In [ ]:
src_dir = '/content/Dataset/test/images'
query_dir = '/content/Dataset/queries/queries'

submission = pd.read_csv('/content/Dataset/sample_submission.csv')

model_name = "openai/clip-vit-large-patch14"

In [ ]:
model = AutoModel.from_pretrained(model_name).cuda().eval()
processor = CLIPProcessor.from_pretrained(model_name)

In [ ]:
query_images = []
query_classes = []

with torch.no_grad():
  for file in os.listdir(query_dir):
    image_path = os.path.join(query_dir, file)
    outputs = process_image(image_path, model, processor)
    query_images.append(outputs)
    query_classes.append(int(file[:-4]))

    query_images = torch.cat(query_images)

In [ ]:
submission['cosine_class'] = 22

for idx, row in tqdm(submission.iterrows()):

    image_path = os.path.join(src_dir, row['img_file'])
    result_class = find_top_match(image_path, model, processor, query_images, query_classes)
    if result_class:
        submission.at[idx, 'cosine_class'] = result_class


sub = submission[['img_file', 'cosine_class']]
sub.columns = ['img_file', 'class']
sub.to_csv('submission.csv', index=False)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.

1120it [08:13,  2.27it/s]


In [ ]:
sub.iloc[5]

img_file    e7ceaec2-8a7c-45d4-a762-95d4456d4413.jpg
class                                             22
Name: 5, dtype: object

In [ ]:
sub